## Setup

In [2]:
# Setup
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor
import os
import numpy as np
import arcpy
from soil_conservation import *
workspace =r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
draftworkspace =r'C:\Users\snewsome\Documents\GitHub\ThresholdEvaluation\2023\WaterQuality' 
# make sql database connection with pyodbc
#conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=sql12;DATABASE=sde_tabular;UID=sde;PWD=staff')

## Soil Conservation

In [ ]:
dfImpChg.GISAcre.sum()
# group by Surface and sum
df = dfImpChg.groupby('Surface')["GISAcre"].sum()

In [ ]:
# get land capability data
dfImpChg, dfImp2019 = get_soil_conservation_data_web()

df = dfImpChg.astype({"Land_Capab": str})
df = df.rename(columns={'Land_Capab':'Land Capability', 'GISAcre':'Total Acres', 'OWNERSHIP_': 'Ownership'})
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df[df['Land Capability'].notna()]
df.set_index('Land Capability')
dfLCType = df.groupby("Land Capability")["Total Acres"].sum().reset_index()

df = dfLCType.replace(to_replace='None', value=np.nan).dropna()

# 2019 analysis results
df = dfImp2019.astype({"Land_Capab": str})
df = df.rename(columns={'Land_Capab':'Land Capability', 'GISAcre':'Acre'})
df.set_index('Land Capability')

# trails were drawn to wide, reduce acreage by 50%
df.loc[(df['Surface']=='Soft')&(df['Feature']=='Trail'), 'Acre'] = df.Acre * 0.5 

# pivot land capbility by acres of surface type
pivotSoilImp = pd.pivot_table(df,index=['Land Capability'],
                            columns='Surface',
                            values=['Acre'], 
                            aggfunc=np.sum,fill_value=0)

flattened = pd.DataFrame(pivotSoilImp.to_records())

df = flattened.rename(columns={"('Acre', 'Hard')":'Acres of Hard Surface 2019',
                            "('Acre', 'Soft')":'Acres of Soft Surface 2019'})
df
# # replace all spaces ond blanks with NaN
# df = df.replace(r'^\s*$', np.nan, regex=True)
# df = df[df['Land Capability'].notna()]

# # calculate acres of coverage
# df["Acres of Coverage 2019"]= df["Acres of Hard Surface 2019"]+df["Acres of Soft Surface 2019"]

# # merge grouped land capability data frame with impervious pivot data frame
# dfMerge = pd.merge(df, dfLCType, on='Land Capability')

# # rename field
# df = dfMerge.rename(columns={'Acre':'Total Acres'})

# # calculate perent coverage
# df['Percent Hard 2019'] = (df['Acres of Hard Surface 2019']/df['Total Acres'])*100
# df['Percent Soft 2019'] = (df['Acres of Soft Surface 2019']/df['Total Acres'])*100
# df['Percent Impervious 2019'] = ((df['Acres of Hard Surface 2019']+df['Acres of Soft Surface 2019'])/df['Total Acres'])*100

# # record percent allowed field
# df['Threshold Value'] = "0%"
# df.loc[df['Land Capability'].isin(['1A','1B','1C','2']), 'Threshold Value'] = "1%"
# df.loc[df['Land Capability'].isin(['3']), 'Threshold Value'] = "5%"
# df.loc[df['Land Capability'].isin(['4']), 'Threshold Value'] = "20%"
# df.loc[df['Land Capability'].isin(['5']), 'Threshold Value'] = "25%"
# df.loc[df['Land Capability'].isin(['6','7']), 'Threshold Value'] = "30%"

# # determine acres of coverage allowed per land capability
# df['Threshold Acres'] = 0
# df.loc[df['Land Capability'].isin(['1A','1B','1C','2']), 'Threshold Acres'] = df['Total Acres']*0.01
# df.loc[df['Land Capability'].isin(['3']), 'Threshold Acres'] = df['Total Acres']*0.05
# df.loc[df['Land Capability'].isin(['4']), 'Threshold Acres'] = df['Total Acres']*0.2
# df.loc[df['Land Capability'].isin(['5']), 'Threshold Acres'] = df['Total Acres']*0.25
# df.loc[df['Land Capability'].isin(['6','7']), 'Threshold Acres'] = df['Total Acres']*0.3

# df2019 = df.drop(columns=['Threshold Acres', 'Threshold Value', 'Total Acres'])

# df = df[['Land Capability',
#         'Acres of Hard Surface 2019',
#         'Acres of Soft Surface 2019',
#         'Acres of Coverage 2019',
#         'Percent Hard 2019',
#         'Percent Soft 2019',
#         'Percent Impervious 2019',
#         'Total Acres',
#         'Threshold Value',
#         'Threshold Acres']]

# df.dropna(subset=['Land Capability'], inplace=True)

# df['Land Capability']= pd.Categorical(df['Land Capability'], ['1A', '1B', '1C', '2', '3', '4', '5', '6', '7'])

# df.sort_values(by="Land Capability")
# df.set_index('Land Capability')

In [ ]:
dfNewImp = pd.read_csv(r"data\raw_data\FinalCoverageChanges_2020-2023.csv")

# fill any NaN values with 0
dfNewImp.fillna(0, inplace=True)
# drop parcle and jurisdiction columns
dfNewImp.drop(['Parcel', 'Jurisdiction'], axis=1, inplace=True)
# group by
df = dfNewImp.groupby(['Bailey1a', 'Bailey1b', 'Bailey1c', 'Bailey2', 'Bailey3', 'Bailey4', 'Bailey5', 'Bailey6', 'Bailey7']).sum()
df = df.reset_index()
# drop total column
# df.drop('Total', axis=1, inplace=True)
# stack the dataframe
df = df.stack().reset_index()
# # rename columns
df.rename(columns={'level_1':'LandCapability', 0:'SqFt'}, inplace=True)
# # drop columns
df.drop(['level_0'], axis=1, inplace=True)
# pivot the dataframe
pivot = pd.pivot_table(df,index=['LandCapability'],
                              values='SqFt', aggfunc=np.sum)
# flatten pivot
flattened = pd.DataFrame(pivot.to_records())
# create acres column
flattened['Acres'] = flattened['SqFt']/43560
# to csv
flattened.to_csv(r"data/processed_data/LandCapability_Acres.csv")

## Noise

In [ ]:
# Setup
#import pandas as pd
#import os
#from arcgis import GIS
#from arcgis.features import FeatureLayer
# Import geopandas
#import geopandas as gpd
#from shapely.geometry import Point
#import arcpy
#from arcgis.features import GeoAccessor


#workspace =r'C:\Users\snewsome\Documents\ArcGIS\Projects\Data Management 2023\Scratch.gdb'

## Update SDE.Monitoring.SDE.Plan_Area_Noise with new data 2020-2023
#excel_file_path = r"F:\Research and Analysis\Noise\Monitoring\CNEL - Plan Areas\ALL_CNEL_PLANAREAS_RESULTS_(1991-2023).xlsx"

# Read the Excel file into a DataFrame
#Noise23_df = pd.read_excel(excel_file_path)

# Keep only columns that you want
#columns_to_keep = ['CNEL LAND USE', '1987 PAS#', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'PAS_NAME', 'LAND_USE', '2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']
#Noise23_df = Noise23_df[columns_to_keep]

#Noise23_df.columns = Noise23_df.columns.str.strip()

# PASNAME
#Noise23_df['PAS_NAME'] = Noise23_df['PAS_NAME'].astype(str)

# Assuming df is your DataFrame
#meltcolumns_to_keep = ['CNEL LAND USE', '1987 PAS#', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE']
#id_columns = ['PAS_NAME']

# Specify the columns to melt (exclude the identifier column)
#columns_to_melt = ['2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']

#id_vars = id_columns + meltcolumns_to_keep

# Use pd.melt to reshape the DataFrame
#Noise23_df = pd.melt(Noise23_df, id_vars=id_vars, value_vars=columns_to_melt, var_name='Year', value_name='CNEL_Maximum_Day_Value')

#print(Noise23_df.head())
#print(Noise23_df.columns)

# Extract only the year from the 'YEAR_OF_COUNT' column and drop "CNEL Maximum Day" from each value
#Noise23_df['Year'] = Noise23_df['Year'].str.replace(' CNEL Maximum Day', '')



# Add 'Category' column
#Noise23_df['Category'] = 'CNEL Maximum Day'

# Drop rows where 'CNEL_Maximum_Day_Value' is NaN
#Noise23_df = Noise23_df.dropna(subset=['CNEL_Maximum_Day_Value'])

# Use lookup_dict to fill in LTINFO# Create a lookup dictionary to fill in LTINFO and ID
#gis = GIS()


#def get_fs_data(service_url):
 #   feature_layer = FeatureLayer(service_url)
  #  query_result = feature_layer.query()
   # feature_list = query_result.features
    #all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    #return all_data


# REST SERVICE data for lookup dictionary
#service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/6'

#dfplannoisesde = get_fs_data(service_url)

#dfplannoisesde.info()
#columnstokeep = ['SITE_NAME', 'ID', 'LTINFO']
#dfplannoisesde = dfplannoisesde.loc[:, columnstokeep]

# Drop duplicates based on 'SITE_NAME' and keep the first occurrence
#unique_values = dfplannoisesde.drop_duplicates(subset='SITE_NAME', keep='first')

# Select specific columns for lookup
#selected_columns = ['SITE_NAME', 'ID', 'LTINFO']

# Convert selected columns to dictionary
#lookup_dict = unique_values[selected_columns].set_index('SITE_NAME').to_dict(orient='index')

# Display the dictionary
#print(lookup_dict)

# Create new columns 'ID' and 'LTINFO' based on 'PAS_NAME' and look up dictionary
#Noise23_df['ID'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('ID'))
#Noise23_df['LTINFO'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('LTINFO'))

# Specify the columns to keep in Final_sdf
#finalcolumns_to_keep = ['PAS_NAME', 'Year', 'CNEL_Maximum_Day_Value', 'Category','ID', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE', 'CNEL LAND USE', 'LTINFO', '1987 PAS#']

# Create a new DataFrame Final_df with the selected columns
#Final_sdf = Noise23_df[finalcolumns_to_keep]

# Create a GeoDataFrame by converting 'LATITUDE' and 'LONGITUDE' to Point geometry
#geometry = [Point(xy) for xy in zip(Final_sdf['MONITORING SITE LONGITUDE'], Final_sdf['MONITORING SITE LATITUDE'])]
#Final_sdf = gpd.GeoDataFrame(Final_sdf, geometry=geometry)

# Field mapping so that columns match
#field_mapping = {
 #   '1987 PAS#': 'PAS_1987',
  #  'ID': 'ID',
   # 'MONITORING SITE LATITUDE': 'LATITUDE',
    #'MONITORING SITE LONGITUDE': 'LONGITUDE',
    #'PAS_NAME': 'SITE_NAME',
    #'LAND_USE': 'LAND_USE',
    #'CNEL LAND USE': 'CNEL_LAND_USE',
    #'Category': 'CATEGORY',
    #'Year': 'YEAR_OF_COUNT',
    #'CNEL_Maximum_Day_Value': 'COUNT_VALUE',
    #'LTINFO': 'LTINFO'
#}

# Create a new DataFrame Final_df with columns based on the field mapping
#Final_sdf = Final_sdf.rename(columns=field_mapping)

# Convert GeoPandas DataFrame to ArcGIS Spatially Enabled DataFrame
#sedf = GeoAccessor.from_geodataframe(Final_sdf)

# Display the updated Final_df
#print(list(Final_sdf.columns))

# Save Final_df to a CSV file
#Final_sdf.to_csv(r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Drafts\NoiseThreshold23.csv', index=False)

# Convert the SEDF to a feature class
#sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise23staging'), 
 #                                                           sanitize_columns=False)



#### CNEL Average Update SDE.Tabular SDE.Thresholdevaluation_PlanAreaNoise

In [ ]:
# update SDE.Tabular.sde.thresholdevaluation_PlanAreaNoise---Average CNEL over timeframe
#updated to make a feature class instead of a csv
# Setup
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor
import os
import numpy as np


workspace =r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# Update SDE.tabular.SDE.threshhold_PlanAreaNoise with new data 2020-2023
excel_file_path = r"F:\Research and Analysis\Noise\Monitoring\CNEL - Plan Areas\ALL_CNEL_PLANAREAS_RESULTS_(1991-2023).xlsx"

#update this Excel 
Excel_Final = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Noise\ThresholdData_PlanAreaNoise.xlsx"
# Read the Excel file into a DataFrame
Noise23_df = pd.read_excel(excel_file_path)

# Keep only columns that you want
columns_to_keep = ['PAS_NAME', 'CNEL LAND USE', '2023 CNEL Average', '2022 CNEL Average', '2021 CNEL Average', '2020 CNEL Average', 'CNEL limit']
Noise23_df = Noise23_df[columns_to_keep]

Noise23_df.columns = Noise23_df.columns.str.strip()



# melted columns to keep
meltcolumns_to_keep = ['CNEL LAND USE']
id_columns = ['PAS_NAME']

# Specify the columns to melt (exclude the identifier column)
columns_to_melt = ['2023 CNEL Average', '2022 CNEL Average', '2021 CNEL Average', '2020 CNEL Average']

id_vars = id_columns + meltcolumns_to_keep

# Use pd.melt to reshape the DataFrame
Noise23_df = pd.melt(Noise23_df, id_vars=id_vars, value_vars=columns_to_melt, var_name='Year', value_name='CNEL_Average')

# Extract only the year from the 'YEAR_OF_COUNT' column and drop "CNEL Maximum Day" from each value
#Noise23_df['Year'] = Noise23_df['Year'].str.replace(' CNEL Average', '')
# Extract only the year from the 'Year' column and drop "CNEL Average" from each value
Noise23_df['Year'] = Noise23_df['Year'].str.extract('(\d+)').astype(int)

# Drop rows where 'CNEL_Average' is NaN
Noise23_df = Noise23_df.dropna(subset=['CNEL_Average'])

print(Noise23_df.head())
print(Noise23_df.columns)


# Calculate the average for each category
Value = Noise23_df.groupby('CNEL LAND USE')['CNEL_Average'].agg(np.mean)

# Create new DataFrame with category averages
Noiseavg_df = pd.DataFrame(Value).reset_index()
Noiseavg_df.columns = ['CNEL LAND USE', 'Value']

# Create new rows with the calculated averages and the desired year range
year_range = '2020-2023'  # Define the desired year range

Noiseavg_df['Year'] = '2020-2023'
Noiseavg_df['Description'] = 'Average CNEL of all sites during timeframe'

#Defining Threshold Value by category
def categorize_value(row):
    if row['CNEL LAND USE'] == 'COMMERCIAL':
        return '60'
    elif row['CNEL LAND USE'] == 'HIGH DENSITY RESIDENTIAL':
        return '55'
    elif row['CNEL LAND USE'] == 'HOTEL/MOTEL':
        return '60'
    elif row['CNEL LAND USE'] == 'INDUSTRIAL':
        return '65'
    elif row['CNEL LAND USE'] == 'LOW DENSITY RESIDENTIAL':
        return '50'
    elif row['CNEL LAND USE'] == 'URBAN OUTDOOR RECREATION':
        return '55'
    elif row['CNEL LAND USE'] == 'RURAL OUTDOOR RECREATION':
        return '50'
    elif row['CNEL LAND USE'] == 'WILDERNESS/ROADLESS':
        return '45'

    else:
        return '45'

# Apply the function to each row and assign the result to a new column 'Category_Type'
Noiseavg_df['Threshold_Value'] = Noiseavg_df.apply(categorize_value, axis=1)

#Defining Threshold Value by category
def categorize_value(row):
    if row['CNEL LAND USE'] == 'COMMERCIAL':
        return 'Commercial Areas'
    elif row['CNEL LAND USE'] == 'HIGH DENSITY RESIDENTIAL':
        return 'High Density Residential'
    elif row['CNEL LAND USE'] == 'HOTEL/MOTEL':
        return 'Hotel / Motel Areas'
    elif row['CNEL LAND USE'] == 'INDUSTRIAL':
        return 'Industrial Areas'
    elif row['CNEL LAND USE'] == 'LOW DENSITY RESIDENTIAL':
        return 'Low Density Residential'
    elif row['CNEL LAND USE'] == 'URBAN OUTDOOR RECREATION':
        return 'Urban Outdoor Recreation Areas'
    elif row['CNEL LAND USE'] == 'RURAL OUTDOOR RECREATION':
        return 'Rural Outdoor Recreation Areas'
    elif row['CNEL LAND USE'] == 'WILDERNESS/ROADLESS':
        return 'Wilderness and Roadless'

    else:
        return 'Critical Wildlife Habitat'

# Apply the function to each row and assign the result to a new column 'Category_Type'
Noiseavg_df['Category'] = Noiseavg_df.apply(categorize_value, axis=1)

print(Noiseavg_df)


# Specify the columns to keep in Final_sdf
finalcolumns_to_keep = [ 'Year', 'Threshold_Value','Description', 'Category', 'Value']

# Create a new DataFrame Final_df with the selected columns
df = Noiseavg_df[finalcolumns_to_keep]



# Field mapping so that columns match
field_mapping = {
    'Threshold_Value': 'Threshold_Value',
    'Category': 'Category',
    'Description': 'Description',
    'Year': 'Year',
    'CNEL_Average': 'Value',
    
}

# Create a new DataFrame Final_df with columns based on the field mapping
df = df.rename(columns=field_mapping)



## Update SDE.Monitoring.sde.Plan_Area_Noise for Monitoring Dashboard

In [ ]:
#updated to make a feature class instead of a csv
# Setup
import pandas as pd
from arcgis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor
import os


workspace =r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'

# Update SDE.Monitoring.SDE.Plan_Area_Noise with new data 2020-2023
excel_file_path = r"F:\Research and Analysis\Noise\Monitoring\CNEL - Plan Areas\ALL_CNEL_PLANAREAS_RESULTS_(1991-2023).xlsx"

# Read the Excel file into a DataFrame
Noise23_df = pd.read_excel(excel_file_path)

# Keep only columns that you want
columns_to_keep = ['CNEL LAND USE', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'PAS_NAME', 'LAND_USE', '2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']
Noise23_df = Noise23_df[columns_to_keep]

Noise23_df.columns = Noise23_df.columns.str.strip()

# PASNAME
Noise23_df['PAS_NAME'] = Noise23_df['PAS_NAME'].astype(str)

# melted columns to keep
meltcolumns_to_keep = ['CNEL LAND USE', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE']
id_columns = ['PAS_NAME']

# Specify the columns to melt (exclude the identifier column)
columns_to_melt = ['2023 CNEL Maximum Day', '2022 CNEL Maximum Day', '2021 CNEL Maximum Day', '2020 CNEL Maximum Day']

id_vars = id_columns + meltcolumns_to_keep

# Use pd.melt to reshape the DataFrame
Noise23_df = pd.melt(Noise23_df, id_vars=id_vars, value_vars=columns_to_melt, var_name='Year', value_name='CNEL_Maximum_Day_Value')

print(Noise23_df.head())
print(Noise23_df.columns)

# Extract only the year from the 'YEAR_OF_COUNT' column and drop "CNEL Maximum Day" from each value
Noise23_df['Year'] = Noise23_df['Year'].str.replace(' CNEL Maximum Day', '')



# Add 'Category' column
Noise23_df['Category'] = 'CNEL Maximum Day'

# Drop rows where 'CNEL_Maximum_Day_Value' is NaN
Noise23_df = Noise23_df.dropna(subset=['CNEL_Maximum_Day_Value'])

# Use lookup_dict to fill in LTINFO# Create a lookup dictionary to fill in LTINFO and ID
gis = GIS()


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    feature_list = query_result.features
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data


# REST SERVICE data for lookup dictionary
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/6'

dfplannoisesde = get_fs_data(service_url)

dfplannoisesde.info()
columnstokeep = ['SITE_NAME', 'ID', 'LTINFO', 'PAS_1987', 'LATITUDE', 'LONGITUDE']
dfplannoisesde = dfplannoisesde.loc[:, columnstokeep]

# Drop duplicates based on 'SITE_NAME' and keep the first occurrence
unique_values = dfplannoisesde.drop_duplicates(subset='SITE_NAME', keep='first')

# Select specific columns for lookup
selected_columns = ['SITE_NAME', 'ID', 'LTINFO', 'PAS_1987', 'LATITUDE', 'LONGITUDE']

def update_lat_lon_from_lookup_dict(noise_data, lookup_dict):
    # Assuming 'SITE_NAME' is the common identifier between DataFrame and lookup dictionary
    for index, row in noise_data.iterrows():
        # Get the 'LATITUDE' and 'LONGITUDE' from the lookup dictionary using the 'SITE_NAME' from the DataFrame
        lat_lon = lookup_dict.get(row['PAS_NAME'])
        # Update the 'MONITORING SITE LATITUDE' and 'MONITORING SITE LONGITUDE' in the DataFrame with the 'LATITUDE' and 'LONGITUDE' from the lookup dictionary
        if lat_lon is not None:
            noise_data.at[index, 'MONITORING SITE LATITUDE'] = lat_lon['LATITUDE']
            noise_data.at[index, 'MONITORING SITE LONGITUDE'] = lat_lon['LONGITUDE']

    
    return noise_data

# Convert selected columns to dictionary
lookup_dict = unique_values[selected_columns].set_index('SITE_NAME').to_dict(orient='index')

# Display the dictionary
print(lookup_dict)

# Update 'MONITORING SITE LATITUDE' and 'MONITORING SITE LONGITUDE' in Noise23_df with data from the lookup dictionary
Noise23_df = update_lat_lon_from_lookup_dict(Noise23_df, lookup_dict)



# Create new columns 'ID' and 'LTINFO' based on 'PAS_NAME' and look up dictionary
Noise23_df['ID'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('ID'))
Noise23_df['LTINFO'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('LTINFO'))
Noise23_df['PAS_1987'] = Noise23_df['PAS_NAME'].apply(lambda x: lookup_dict.get(x, {}).get('PAS_1987'))

# Specify the columns to keep in Final_sdf
finalcolumns_to_keep = ['PAS_NAME', 'Year', 'CNEL_Maximum_Day_Value', 'Category','ID', 'MONITORING SITE LATITUDE', 'MONITORING SITE LONGITUDE', 'LAND_USE', 'CNEL LAND USE', 'LTINFO', 'PAS_1987']

# Create a new DataFrame Final_df with the selected columns
df = Noise23_df[finalcolumns_to_keep]

# Field mapping so that columns match
field_mapping = {
    'PAS_1987': 'PAS_1987',
    'ID': 'ID',
    'MONITORING SITE LATITUDE': 'LATITUDE',
    'MONITORING SITE LONGITUDE': 'LONGITUDE',
    'PAS_NAME': 'SITE_NAME',
    'LAND_USE': 'LAND_USE',
    'CNEL LAND USE': 'CNEL_LAND_USE',
    'Category': 'CATEGORY',
    'Year': 'YEAR_OF_COUNT',
    'CNEL_Maximum_Day_Value': 'COUNT_VALUE',
    'LTINFO': 'LTINFO'
}

# Create a new DataFrame Final_df with columns based on the field mapping
df = df.rename(columns=field_mapping)

# Convert DataFrame to Spatially Enabled DataFrame
sedf = GeoAccessor.from_xy(df, x_column='LONGITUDE', y_column='LATITUDE')

# Convert the SEDF to a feature class without sanitizing columns
sedf.spatial.to_featureclass(location=os.path.join(workspace, 'PlanAreaNoise_23_Staging'), sanitize_columns=False)


In [ ]:
#reproject feature class 4236 to 26910
# Set the input and output paths
arcpy.env.workspace=r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
input_feature_class = 'PlanAreaNoise_23_Staging'
output_feature_class = 'PlanAreaNoise_23_ready'
 
# Specify the target coordinate system (EPSG code 26910 for UTM Zone 10N)
target_coordinate_system = arcpy.SpatialReference(26910)
 
# Use the Project tool to reproject the feature class
arcpy.management.Project(
    input_feature_class,
    output_feature_class,
    target_coordinate_system
)
 
print(f"Reprojection completed. Output feature class: {output_feature_class}")

In [ ]:
arcpy.Delete_management(r'c:\Users\snewsome\Documents\GitHub\ThresholdEvaluation\Scripts\2023\db_connect\ConnectionFile.sde')

In [ ]:
import arcpy
from time import strftime


#push data to sde
#Create database connection
inWorkspace = r"F:\Research and Analysis\Workspace\Sarah\SarahVector.sde"
arcpy.env.workspace = inWorkspace

# Specify the name of the new version and the parent version
new_version_name = "PlanAreaNoise_" + strftime("%Y-%m-%d")
parent_version = "sde.DEFAULT"
version_name_full = '"TAHOE\SNEWSOME".'+ new_version_name

# List all versions in the geodatabase
existing_versions = [version.name for version in arcpy.da.ListVersions(inWorkspace)]

In [ ]:
import arcpy
from time import strftime


#push data to sde
#Create database connection
inWorkspace=r"F:\Research and Analysis\Workspace\Sarah\SarahVector.sde"
arcpy.env.workspace=inWorkspace

# Specify the name of the new version and the parent version
new_version_name = "PlanAreaNoise_" + strftime("%Y-%m-%d")
parent_version = "sde.DEFAULT"
version_name_full = '"TAHOE\SNEWSOME".'+ new_version_name

# List all versions in the geodatabase
existing_versions = [version.name for version in arcpy.da.ListVersions(inWorkspace)]

# Check if the specified version exists
if version_name_full in existing_versions:
    # Delete the version
    arcpy.management.DeleteVersion(inWorkspace, version_name_full)
    print(f"Version '{version_name_full}' deleted successfully.")
else:
    print(f"Version '{version_name_full}' does not exist.")

# Create a new version
arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")

# If you want to create a connection file, you can use the following code:
arcpy.CreateDatabaseConnection_management(
    out_folder_path='db_connect/',
    out_name="ConnectionFile.sde",
    database_platform="SQL_SERVER",  
    instance='sql12',
    database='sde',
    account_authentication="OPERATING_SYSTEM_AUTH",  
    version_type='TRANSACTIONAL',
    version=version_name_full
)

PlanNoisesde=os.path.join(version_name_full, "SDE.Monitoring\SDE.Plan_Area_Noise")
PlanAreaNoise = 'planareanoise'
arcpy.MakeFeatureLayer_management(PlanNoisesde, PlanAreaNoise)


# Create a new version
#arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")
arcpy.ChangeVersion_management(PlanAreaNoise, 'TRANSACTIONAL', version_name_full, '')

#Start
edit = arcpy.da.Editor('db_connections/ConnectionFile.sde')
edit.startEditing(False, True)

# Append the records from the temporary feature class to the target feature class
arcpy.management.Append(
    inputs='PlanAreaNoise_23_ready',
    target=PlanNoisesde, 
    schema_type='NO_TEST'
)

#Stop
edit.stopEditing(True)



In [ ]:
try:
    arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")
    print("Version created successfully.")
except arcpy.ExecuteError as e:
    print(f"Error: {e}")

In [ ]:
#Start
edit = arcpy.da.Editor('db_connections/ConnectionFile.sde')
edit.startEditing(False, True)

# Append the records from the temporary feature class to the target feature class
arcpy.management.Append(
    inputs='PlanAreaNoise_23_ready',
    target=os.path.join(version_full_name, "SDE.Monitoring\SDE.Plan_Area_Noise"), 
    schema_type='NO_TEST'
)

#Stop
edit.stopEditing(True)

In [ ]:
arcpy.Delete_management("planareanoise")

In [ ]:
# Start an edit session
edit = arcpy.da.Editor('db_connections/ConnectionFile.sde')
edit.startEditing(False, True)

    # Create a new version
arcpy.CreateVersion_management(inWorkspace, parent_version, new_version_name, "PUBLIC")

#### Shorezone Noise

## Stream/CSCI

In [ ]:
#Calculate the Average CSCI score for TPAandTPBsites for 2020-2022 and fill in csv, Next code block will be splitting TPA and TPB into two different avg scores
#Setup
#Create Dictionary Usring Rest Service data

import pandas as pd
import os
from arcgis import GIS
from arcgis.features import FeatureLayer
import numpy as np
import requests
import arcpy

gis = GIS()
# Connect to TRPA Enterprise GIS Portal *if it's a service only shared with org
# portal_user = "TRPA_PORTAL_ADMIN"
# portal_pwd = str(os.environ.get('Password'))
# portal_url = "https://maps.trpa.org/portal/"
wk_memory = "memory" + "\\"

#Set up Workspace
# set workspace and sde connections 
working_folder = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries"
workspace      = "C:\GIS\Scratch.gdb"
arcpy.env.workspace = "C:\GIS\Scratch.gdb"

# network path to connection files
filePath = r"C:\\GIS\\DB_CONNECT"


## CSV to be updated with new average
StreamAvg = os.path.join(working_folder,"ThresholdData_Stream.csv")


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

# Service URL
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'

# Get Stream data as a Spatially Enabled DataFrame
sdfStreamHab = get_fs_data(service_url)

# Keep only necessary columns
columnstokeep = ['SITE_NAME', 'STATION_TYPE', 'DURATION', 'LATITUDE', 'LONGITUDE', 'LTINFO', 'COUNT_VALUE', 'YEAR_OF_COUNT']
sdfStreamHab = sdfStreamHab.loc[:, columnstokeep]

# Query only 2020-2022 data
sdfStreamHab['Date'] = pd.to_datetime(sdfStreamHab['YEAR_OF_COUNT'], format='%Y')
filtered_df = sdfStreamHab[
    (sdfStreamHab['Date'].dt.year.between(2020, 2022)) & 
    (sdfStreamHab['SITE_NAME'].str.contains('TPB|TPA'))
].copy()

# Calculate the average of the 'COUNT_VALUE' column
AvgCSCI23 = filtered_df['COUNT_VALUE'].mean()

# Update CSV ThresholdData_Stream.csv
csv_path = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv"
existing_df = pd.read_csv(csv_path)

Description = "Average CSCI score of all trend sites (48 sites)"
Year = '2020-2022'
Value = AvgCSCI

existing_entry = existing_df[
    (existing_df['Description'] == Description) &
    (existing_df['Year'] == Year)
]

if existing_entry.empty:
    new_entry = {'Description': Description, 'Year': Year, 'Value': Value}
    existing_df = pd.concat([existing_df, pd.DataFrame([new_entry])], ignore_index=True)


existing_df.to_csv(csv_path, index=False)
print(f"CSV file '{csv_path}' has been updated.")



#### Avg Stream CSCI split into TPB and TPA for all years. in a new excel sheet

In [ ]:
import pandas as pd
import os
from arcgis import GIS
from arcgis.features import FeatureLayer
import numpy as np

import arcpy

gis = GIS()

wk_memory = "memory" + "\\"

#Set up Workspace
# set workspace and sde connections 
working_folder = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Fisheries"
workspace      = "F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb"
arcpy.env.workspace = "F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb"



## excel to be updated with new averages split into TPA and TPB
StreamAvg = os.path.join(working_folder,"ThresholdData_StreamsTavg.xlsx")


def get_fs_data(service_url):
    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features
    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

# Service URL
service_url = 'https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/8'

# Get Stream data as REST SERVICE to a Spatially Enabled DataFrame
sdfStreamHab = get_fs_data(service_url)

# Keep only necessary columns
columnstokeep = ['SITE_NAME', 'STATION_TYPE', 'DURATION', 'LATITUDE', 'LONGITUDE', 'LTINFO', 'COUNT_VALUE', 'YEAR_OF_COUNT']
sdfStreamHab = sdfStreamHab.loc[:, columnstokeep]

#This is where the magic calculations happen for each threshold
# Mke year of count into date format
sdfStreamHab['Date'] = pd.to_datetime(sdfStreamHab['YEAR_OF_COUNT'], format='%Y')

# Define bins for grouping by year ranges
bins = [2010, 2012, 2014, 2016, 2020, 2023]

# Define labels for the bins
labels = ['2010-2011', '2012-2013', '2014-2015', '2016-2019', '2020-2022']

# Create a new column 'YearRange' to group the data by year ranges
sdfStreamHab['YearRange'] = pd.cut(sdfStreamHab['Date'].dt.year, bins=bins, labels=labels, right=False)

# Filter the DataFrame for TPB sites and TPA sites
filtered_df_tpb = sdfStreamHab[sdfStreamHab['SITE_NAME'].str.contains('TPB')]
filtered_df_tpa = sdfStreamHab[sdfStreamHab['SITE_NAME'].str.contains('TPA')]

# Group by 'YearRange', then calculate the average 'COUNT_VALUE' TPB sites
averages_tpb = filtered_df_tpb.groupby('YearRange')['COUNT_VALUE'].mean().reset_index()
averages_tpb.rename(columns={'COUNT_VALUE': 'Average_COUNT_VALUE_TPB'}, inplace=True)

averages_tpa = filtered_df_tpa.groupby('YearRange')['COUNT_VALUE'].mean().reset_index()
averages_tpa.rename(columns={'COUNT_VALUE': 'Average_COUNT_VALUE_TPA'}, inplace=True)

# Create Description and Year columns in both DataFrames
averages_tpb['Description'] = 'Average CSCI Score per Trend Panel(24 Sites)'
averages_tpa['Description'] = 'Average CSCI Score per Trend Panel(24 Sites)'
averages_tpb['Year'] = averages_tpb['YearRange']
averages_tpa['Year'] = averages_tpa['YearRange']

# Merge the two DataFrames on 'Year' and 'Description' to create the final merged DataFrame
FAdf = pd.merge(averages_tpb, averages_tpa, on=['Year', 'Description'], how='outer')
# Sort by 'Year' for clarity
FAdf.sort_values('Year', inplace=True)

# Reset index for clarity
FAdf.reset_index(drop=True, inplace=True)

# Define columns to keep
columns_to_keep = ['Year', 'Description', 'Average_COUNT_VALUE_TPB', 'Average_COUNT_VALUE_TPA']

# Select only the desired columns
FAdf = FAdf[columns_to_keep]
# Print the combined averages
print(FAdf)
print("FAdf:")

# Write the updated DataFrame back to the CSV file
FAdf.to_excel(StreamAvg, index=False)

print(f"The excel file '{StreamAvg}' has been updated with new averages successfully.")

In [ ]:
#Use F:Research and Analysis/Threshold reporting\ThresholdData\Fisheries\ThresholdData_Streams.csv' to update SDE.tabular.....table


In [ ]:
#Calculate Rating or CSCI values and then find the percent of total streams in each rating
excel_file_path="F:\Research and Analysis\Fisheries\Streams\Bioassessment\California Stream Condition Index\California Stream Condition Index\CSCI_Scores_AllSites_AllYears.xlsx"

Alldf = pd.read_excel(excel_file_path, sheet_name='CSCI_Scores_AllSites_AllYears')

#Calculate Rating for CSCI value
#Define a function to categorize CSCI values based on ranges
def categorize_value(value):
    if 0 <= value < 0.8:
        return 'marginal'
    elif 0.8 <= value < 1.0:
        return 'good'
    else:
        return 'excellent'

Alldf['Rating']=Alldf['CSCI_Score'].apply(categorize_value)

print(Alldf)

#Find average for each rating- Poor, Marginal, Good, Excellent
# Group by 'Category' and calculate the mean of 'Count'
average_count_by_category = df.groupby('Rating')['CSCI_Score'].mean()    

## Wild Life

In [ ]:
print(Wildlifedf['Wildlife_Species'].dtype)

### Wildlife Species

In [ ]:
import arcpy
import pandas as pd
import os
# Delete the table

workspace = r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
#arcpy.Delete_management(os.path.join(workspace, 'Wildlife_Staging'))
# Update SDE.tabular.SDE.threshhold_PlanAreaNoise with new data 2020-2023
excel_file_path = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Wildlife\Wildlife_ThresholdData.xlsx"

# Read the Excel file into a DataFrame
Rawdf = pd.read_excel(excel_file_path)

# Grab needed data by year
Wildlifedf = Rawdf.loc[(Rawdf['Year'] >= 2020) & (Rawdf['Year'] <= 2023)].copy()

#Rename columns directly
Wildlifedf.rename(columns={'Wildlife Species': 'Wildlife_Species', 'Threshold Value': 'Threshold_Value', 'Count': 'Total'}, inplace=True)
# Convert 'Wildlife_Species' column to string type
Wildlifedf['Wildlife_Species'] = Wildlifedf['Wildlife_Species'].astype(str)
# Define the fields to include in the table
field_names = ['Wildlife_Species', 'Threshold_Value', 'Category', 'Year', 'Total']  # Add more fields if necessary

# Convert 'Wildlife Species' column to string type to ensure compatibility
Wildlifedf['Wildlife_Species'] = Wildlifedf['Wildlife_Species'].astype(str)

# Define the fields to include in the table with specified field types
field_types = {'Wildlife_Species': 'Text', 'Threshold_Value': 'Text', 'Category': 'TEXT', 'Year': 'Double', 'Total': 'Double'}

# Create an empty table in the workspace
table_path = os.path.join(workspace, 'Wildlife_staging')
arcpy.CreateTable_management(workspace, 'Wildlife_staging')

# Add fields to the table with specified field types
for field_name, field_type in field_types.items():
    arcpy.AddField_management(table_path, field_name, field_type)

# Convert the DataFrame to a structured numpy array
array = Wildlifedf[list(field_types.keys())].to_records(index=False)

# Insert data into the table
with arcpy.da.InsertCursor(table_path, list(field_types.keys())) as cursor:
    for row in array:
        cursor.insertRow(row)


        #GO TO WORKSPACE FOLDER AND APPEND TO F:\GIS\DB_CONNECT\Tabular.sde\SDE.ThresholdEvaluation_Wildlife in PRO

## Waterfowl Human Activity

In [ ]:
# Not done
import arcpy
import pandas as pd
import os
# Delete the table

workspace = r'F:\Research and Analysis\Workspace\Sarah\Data Management 2023\Scratch.gdb'
#arcpy.Delete_management(os.path.join(workspace, 'Wildlife_Staging'))
# Update SDE.tabular.SDE.threshhold_PlanAreaNoise with new data 2020-2023
excel_file_path = r"F:\Research and Analysis\Threshold reporting\ThresholdData\Wildlife\Waterfowl_HumanActivityRating_2019.xlsx"

# Read the Excel file into a DataFrame
Rawdf = pd.read_excel(excel_file_path)

# Grab needed data by year
Wildlifedf = Rawdf.loc[(Rawdf['Year'] >= 2020) & (Rawdf['Year'] <= 2023)].copy()

#Rename columns directly
fowldf.rename(columns={'Wildlife Species': 'Wildlife_Species', 'Threshold Value': 'Threshold_Value', 'Count': 'Total'}, inplace=True)
# Convert 'Wildlife_Species' column to string type
fowldf['Wildlife_Species'] = fowldf['Wildlife_Species'].astype(str)
# Define the fields to include in the table
field_names = ['Wildlife_Species', 'Threshold_Value', 'Category', 'Year', 'Total']  # Add more fields if necessary

# Convert 'Wildlife Species' column to string type to ensure compatibility
Wildlifedf['Wildlife_Species'] = Wildlifedf['Wildlife_Species'].astype(str)

# Define the fields to include in the table with specified field types
field_types = {'Wildlife_Species': 'Text', 'Threshold_Value': 'Text', 'Category': 'TEXT', 'Year': 'Double', 'Total': 'Double'}

# Create an empty table in the workspace
table_path = os.path.join(workspace, 'Wildlife_staging')
arcpy.CreateTable_management(workspace, 'Wildlife_staging')

# Add fields to the table with specified field types
for field_name, field_type in field_types.items():
    arcpy.AddField_management(table_path, field_name, field_type)

# Convert the DataFrame to a structured numpy array
array = Wildlifedf[list(field_types.keys())].to_records(index=False)

# Insert data into the table
with arcpy.da.InsertCursor(table_path, list(field_types.keys())) as cursor:
    for row in array:
        cursor.insertRow(row)

## Wildlife Data confidence interval

In [3]:
#REST Service that refers to ['sde_tabular.SDE.ThresholdEvaluation_Waterfowl']

#['sde_tabular.SDE.ThresholdEvaluation_Wildlife']
def get_fs_data(wildlife_url):
    feature_layer = FeatureLayer(wildlife_url)
    query_result = feature_layer.query()
    feature_list = query_result.features
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])
    return all_data

waterfowl_url = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/94"
wildlife_url = "https://maps.trpa.org/server/rest/services/LTInfo_Monitoring/MapServer/96"
draftworkspace = r"C:\Users\snewsome\Documents\GitHub\ThresholdEvaluation\2023\Wildlife\Chart\Draft"
dfWildlife = get_fs_data(wildlife_url)

## Wintering Bald Eagle linear regression confidence interval

In [12]:
from scipy.stats import linregress, t

# Setup dataframe
df = dfWildlife.loc[dfWildlife['Wildlife_Species'] == 'Bald Eagle - winter']

# Calculate linear regression using scipy's linregress
slope, intercept, r_value, p_value, std_err = linregress(df['Year'], df['Total'])
r_squared = r_value ** 2

# Calculate the degrees of freedom
n = len(df)
dof = n - 2

# Calculate two-tailed t-value for 95% confidence interval
t_value = t.ppf(0.975, dof)  # 0.975 for a two-tailed 95% confidence level

# Calculate confidence interval for the slope
slope_ci_lower = slope - t_value * std_err
slope_ci_upper = slope + t_value * std_err



# Display results
print(f"Slope: {slope}")
print(f"95% Confidence Interval for Slope: [{slope_ci_lower}, {slope_ci_upper}]")
print(f"R-squared: {r_squared}")
print(f"P-value: {p_value}")
print(f"dof: {dof}")

Slope: 0.847179487179487
95% Confidence Interval for Slope: [0.5454479896893918, 1.1489109846695822]
R-squared: 0.5831917257885286
P-value: 5.65430032472803e-06
dof: 24


## Osprey 95 confidence interval of linear regression

In [8]:
from scipy.stats import linregress, t

# setup dataframe
df = dfWildlife.loc[dfWildlife['Wildlife_Species'] == 'Osprey']

# Calculate linear regression using scipy's linregress
slope, intercept, r_value, p_value, std_err = linregress(df['Year'], df['Total'])
r_squared = r_value ** 2

# Calculate the degrees of freedom
n = len(df)
dof = n - 2

# Calculate two-tailed t-value for 95% confidence interval
t_value = t.ppf(0.975, dof)  # 0.975 for a two-tailed 95% confidence level

# Calculate confidence interval for the slope
slope_ci_lower = slope - t_value * std_err
slope_ci_upper = slope + t_value * std_err

# Display results
print(f"Slope: {slope}")
print(f"95% Confidence Interval for Slope: [{slope_ci_lower}, {slope_ci_upper}]")
print(f"R-squared: {r_squared}")
print(f"P-value: {p_value}")

Slope: 0.26678876678876684
95% Confidence Interval for Slope: [0.08086824630055772, 0.45270928727697596]
R-squared: 0.2589110593305362
P-value: 0.006720059967321667


## Falcon 95 confidence interval

In [9]:
from scipy.stats import linregress, t

# setup dataframe
df = dfWildlife.loc[dfWildlife['Wildlife_Species'] == 'Peregrine Falcon']

# Calculate linear regression using scipy's linregress
slope, intercept, r_value, p_value, std_err = linregress(df['Year'], df['Total'])
r_squared = r_value ** 2

# Calculate the degrees of freedom
n = len(df)
dof = n - 2

# Calculate two-tailed t-value for 95% confidence interval
t_value = t.ppf(0.975, dof)  # 0.975 for a two-tailed 95% confidence level

# Calculate confidence interval for the slope
slope_ci_lower = slope - t_value * std_err
slope_ci_upper = slope + t_value * std_err

# Display results
print(f"Slope: {slope}")
print(f"95% Confidence Interval for Slope: [{slope_ci_lower}, {slope_ci_upper}]")
print(f"R-squared: {r_squared}")
print(f"P-value: {p_value}")

Slope: 0.31029411764705883
95% Confidence Interval for Slope: [0.2240225956947095, 0.39656563959940816]
R-squared: 0.8095463223929447
P-value: 2.0861186437493537e-06


### Watercraft Inspections

In [ ]:
import pandas as pd
import plotly.express as px


url = 'https://www.laketahoeinfo.org/WebServices/GetReportedEIPIndicatorProjectAccomplishments/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476/16'

df = pd.read_json(url)
df

# stacked bar chart with plotly
fig = px.bar(df, x='IndicatorProjectYear', y='IndicatorProjectValue', color='PMSubcategoryOption1', barmode='stack')
fig.show()

# Air Quality Data to table in Arc Pro to be appended manually

In [ ]:
#-----------------------#
#Add Data to SDE
#------------------------#
import pandas as pd

# 1. Specify File Paths
file_paths = ["F:\Research and Analysis\Air Quality\Annual Reports DRI\AQ data 2023.xlsx",
              "F:\Research and Analysis\Air Quality\Annual Reports DRI\AQ data 2022.xlsx",
              "F:\Research and Analysis\Air Quality\Annual Reports DRI\AQ data 2021.xlsx",
              "F:\Research and Analysis\Air Quality\Annual Reports DRI\AQ data 2020.xlsx"]

# 2. Read Data from Each File
dfs = []  # List to store DataFrames from each file
sheet_name = 'Indicator Values'  # Name of the sheet to read

for file_path in file_paths:
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    dfs.append(df)

# 3. Concatenate DataFrames
Airdata_df = pd.concat(dfs, ignore_index=True)


#Create layer in arcgis pro to be appended to sde.ThresholdEvaluyation_AirQuality

#Fix up field names

# Rename fields
Airdata_df.rename(columns={'Data?': 'Data', 'Data Source': 'Data_Source'}, inplace=True)
#Add columsn to dataframe
Airdata_df['Threshold_Value']= None
Airdata_df['Percent_of_Threshold_Value']= None
Airdata_df['Include_in_Trend_Analysis']= None
print(Airdata_df)


# Improve code next year , ADD DLBLISS as yes for pm2.5 high24, south lake tahoe sites for pm2.5 avg. etc.
# Convert DataFrame to a list of dictionaries
data = Airdata_df.to_dict(orient='records')

# Get the field names from the field mapping
field_names = list(Airdata_df.columns)


# Create an empty table in the workspace
table_path = os.path.join(workspace, 'Air_staging2')
arcpy.CreateTable_management(workspace, 'Air_staging2')

# Define the fields to include in the table with specified field types
field_types = {'Indicator': 'Text', 'Pollutant': 'Text', 'Statistic': 'Text', 'Year': 'Long', 'Data': 'Text', 'Site': 'Text', 'Value': 'Double', 'Data_Source': 'Text', 'Exceedances': 'Long', 'Threshold_Value': 'Text', 'Percent_of_Threshold_Value': 'Text', 'Include_in_Trend_Analysis': 'Text'}

# Add fields to the table with specified field types
for field_name, field_type in field_types.items():
    arcpy.AddField_management(table_path, field_name, field_type)

# Convert the DataFrame to a structured numpy array
array = Airdata_df[list(field_types.keys())].to_records(index=False)

# Insert data into the table
with arcpy.da.InsertCursor(table_path, list(field_types.keys())) as cursor:
    for row in array:
        cursor.insertRow(row)

print("Data table successfully made.")

# Nearshore Periphyton/Algae

In [ ]:
# set colors
color_discrete_map = {'Incline West': '#33a02c',
                      'Pineland': '#1f78b4', 
                      'Rubicon': '#fb9a99',
                      'Sugarpine': '#911eb4',
                      'Tahoe City': '#ff7f00',
                      'Zephyr': '#cab2d6'                                
                        }
#Import DAta
file_path = r"F:\Research and Analysis\Water Quality Monitoring Program\Nearshore\IntegratedAlgaeMonitoring\data\Peri"

# Load each CSV file into a DataFrame
inclinedf = pd.read_csv(os.path.join(file_path, 'InclineWest_Historic.csv'))
Pinelanddf = pd.read_csv(os.path.join(file_path, 'Pineland_Historic.csv'))
Rubicondf = pd.read_csv(os.path.join(file_path, 'Rubicon_Historic.csv'))
Sugarpinedf = pd.read_csv(os.path.join(file_path, 'Sugarpine_Historic.csv'))
TahoeCitydf = pd.read_csv(os.path.join(file_path, 'TahoeCity_Historic.csv'))
Zephyrdf = pd.read_csv(os.path.join(file_path, 'Zephyr_Historic.csv'))

#Combine All Dataframes
combined_df = pd.concat([inclinedf, Pinelanddf, Rubicondf, Sugarpinedf, TahoeCitydf, Zephyrdf], ignore_index=True)
# Ensure the Date column is in datetime format
combined_df['Sample_Date'] = pd.to_datetime(combined_df['Sample_Date'])

# Extract the year from the Date column and create a new Year column
combined_df['Year'] = combined_df['Sample_Date'].dt.year

#Group by 'Year' and 'Site', then calculate the average of the 'chl' column
grouped_df = combined_df.groupby(['Year', 'site'], as_index=False)['Chl'].mean()

# Preview the resulting DataFrame
print(grouped_df.head())

#May need to drop rows with missing value? not sure how it will show up on chart

In [ ]:


color_discrete_map = {'Incline West': '#008080',
                      'Pineland': '#FF6F61', 
                      'Rubicon Pt.': '#cab2d6',
                      'Sugar Pine Pt.': '#4169E1',
                      'Tahoe City': '#DAA520',
                      'Zephyr Pt.': '#708090'                                
                        }

df = grouped_df


# setup plot
fig = px.line(df, x = 'Year', y= 'Chl', color='site',
                 color_discrete_map = color_discrete_map)

fig.update_traces(hovertemplate='<br>%{y:.2f}')


# set layout
fig.update_layout(title="Nearshore Attached Algae - Average Chlorophyll",
                    font_family=font,
                    template=template,
                    showlegend=True,
                    hovermode="x unified",
                    xaxis = dict(
                        tickmode = 'linear',
                        tick0 = 1985,
                        dtick = 5
                    ),
                    yaxis = dict(
                        tickmode = 'linear',
                        tick0 = 0,
                        dtick = 5,
                        range=[0, 110],
                        title_text='Average Chl'
                    )
                  
                 )

# show figure
fig.show()
# save to HTML
fig.write_html(os.path.join(workspace, "WaterQuality_NearshoreChl.html"))